In [11]:
import os
import json
import logging
import re
from report import Report
import pdb
from collections import defaultdict
from dataclasses import dataclass
from typing import Optional
import vertexai
from vertexai.generative_models import GenerativeModel
from together import Together
import csv

TOGETHER_AI_API_KEY = "368f91cfda9ddef7e1c17a401e54ac7aa16a70900935627505e5beeee1b66c5c"
GCP_PROJECT_ID = "fluent-sprite-424817-q5"

fake_fn = "Datasets/DataSet_Misinfo_FAKE.csv"
true_fn = "Datasets/DataSet_Misinfo_TRUE.csv"
prompt_fn = "LLM_prompt2.txt"

with open(prompt_fn, 'r', encoding='utf-8') as file:
    prompt = file.read()

In [12]:
def query_together_ai(model: str, text: str) -> dict:
        client = Together(api_key=TOGETHER_AI_API_KEY)

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": text}]
        )

        return response.choices[0].message.content


def query_gcp(text: str) -> dict:
    vertexai.init(project=GCP_PROJECT_ID, location="us-central1")
    model = GenerativeModel(model_name="gemini-1.0-pro-002")

    response = model.generate_content(
        text
    )

    return response.text

In [13]:
c70B_fake = 0
c8B_fake = 0
gcp_fake = 0
tot = 0

with open(fake_fn, newline='', encoding='utf-8') as csvfile:
    csvreader= csv.reader(csvfile, delimiter=',')

    # skip header
    next(csvreader)

    for row in csvreader:
        message = row[1]
        model_input = prompt+message

        if len(model_input.split()) > 2500:
            print("yes")
            continue
        try:
            response_70B = query_together_ai("meta-llama/Llama-3-70b-chat-hf", model_input)
            response_8B = query_together_ai("meta-llama/Llama-3-8b-chat-hf", model_input)
            response_gcp = query_gcp(model_input)


            if "misinformation: yes" in response_70B.lower():
                c70B_fake += 1
            if "misinformation: yes" in response_8B.lower():
                c8B_fake += 1
            if "misinformation: yes" in response_gcp.lower():
                gcp_fake += 1
            
            tot += 1

            if tot > 200:
                break
        except:
            continue
        
        

In [14]:
print(tot)
print(c70B_fake)
print(c8B_fake)
print(gcp_fake)

201
8
49
130


In [15]:
c70B_true = 0
c8B_true = 0
gcp_true = 0
tot = 0

with open(true_fn, newline='', encoding='utf-8') as csvfile:
    csvreader= csv.reader(csvfile, delimiter=',')

    # skip header
    next(csvreader)

    for row in csvreader:
        message = row[1]
        model_input = prompt+message

        if len(model_input.split()) > 2500:
            print("yes")
            continue
        try:
            response_70B = query_together_ai("meta-llama/Llama-3-70b-chat-hf", model_input)
            response_8B = query_together_ai("meta-llama/Llama-3-8b-chat-hf", model_input)
            response_gcp = query_gcp(model_input)


            if "misinformation: no" in response_70B.lower():
                c70B_true += 1
            if "misinformation: no" in response_8B.lower():
                c8B_true += 1
            if "misinformation: no" in response_gcp.lower():
                gcp_true += 1
            
            tot += 1

            if tot > 200:
                break
        except:
            continue

In [16]:
print(tot)
print(c70B_true)
print(c8B_true)
print(gcp_true)

201
195
58
139
